In [19]:
import pandas as pd
from statsmodels.regression.mixed_linear_model import MixedLM
import statsmodels.api as sm
from statsmodels.formula.api import ols
from functions.others import choose_dominant_kind, replace_to_categories
from functions.tests.helper import make_decision

Zbiory do analiz wieloczynnikowych:
- szczegółowe wartości
- średnie wartości dla grup A, B

In [84]:
df_answers = pd.read_csv('/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/correctness.csv', index_col='ID')

df_survey = pd.read_csv(
    '/Users/martasolarz/Studies/Thesis/data/survey.csv',
).fillna('NO_VAL')

df_survey = df_survey[df_survey.ID != 6]
df_survey.set_index('ID', inplace=True)

df_survey['MAT'] = df_survey['MAT_roz'].apply(lambda x: x if x == 'NO_VAL' else 'VAL')
df_survey['POL'] = df_survey['JP_roz'].apply(lambda x: x if x == 'NO_VAL' else 'VAL')
df_survey['HIST'] = df_survey['HIST_roz'].apply(lambda x: x if x == 'NO_VAL' else 'VAL')
df_survey['Typ_uczenia'] = df_survey.apply(choose_dominant_kind, axis=1)
df_survey['Kierunek_studiów'] = df_survey['Kierunek_studiów'].apply(
    lambda x: 'MISMaP Geografia' if x == 'Geografia, w ramach studiów międzyobszarowych' else x)
df_survey['Samopocz'] = df_survey['Samopocz'].apply(replace_to_categories)
df_survey['Stres'] = df_survey['Stres'].apply(replace_to_categories)
df_survey['Zmęcz'] = df_survey['Zmęcz'].apply(replace_to_categories)
df_survey['Czyt_map'] = df_survey['Czyt_map'].apply(replace_to_categories)
df_survey['Podziel_uwg'] = df_survey['Podziel_uwg'].apply(replace_to_categories)
df_survey['Analit_myśl'] = df_survey['Analit_myśl'].apply(replace_to_categories)
df_survey['Spostrzeg'] = df_survey['Spostrzeg'].apply(replace_to_categories)
df_survey['Czyt_ze_zroz'] = df_survey['Czyt_ze_zroz'].apply(replace_to_categories)

df_survey.drop(columns=['Daltonizm', 'JP_podst', 'MAT_podst', 'JP_roz', 'MAT_roz', 'FIZ_roz', 'HIST_roz', 'Wzrokowiec',
                        'Słuchowiec', 'Kinestetyk', 'Quest1_after', 'Quest2_after', 'Rank_map', 'Rank_plt', 'Rank_tab',
                        'Rank_txt', 'Data', 'Godzina'], inplace=True)

df_quest = pd.read_csv(
    '/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/quest12.csv',
    usecols=['ID', 'Quest1_before', 'Quest2_before', 'Quest1_after', 'Quest2_after', 'is_change_1', 'is_change_2'],
    index_col='ID')

df_time = pd.read_csv('/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/times.csv', index_col='ID')

df_f_1a = pd.read_csv(
    '/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/fixation/dfs_samples_1a.csv', index_col='ID')
df_f_2a = pd.read_csv(
    '/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/fixation/dfs_samples_2a.csv', index_col='ID')
df_f_3a = pd.read_csv(
    '/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/fixation/dfs_samples_3a.csv', index_col='ID')

df_f_1b = pd.read_csv(
    '/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/fixation/dfs_samples_1b.csv', index_col='ID')
df_f_2b = pd.read_csv(
    '/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/fixation/dfs_samples_2b.csv', index_col='ID')
df_f_3b = pd.read_csv(
    '/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/fixation/dfs_samples_3b.csv', index_col='ID')

df_v_1a = pd.read_csv('/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/visit/dfs_samples_1a.csv',
                      index_col='ID')
df_v_2a = pd.read_csv('/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/visit/dfs_samples_2a.csv',
                      index_col='ID')
df_v_3a = pd.read_csv('/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/visit/dfs_samples_3a.csv',
                      index_col='ID')

df_v_1b = pd.read_csv('/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/visit/dfs_samples_1b.csv',
                      index_col='ID')
df_v_2b = pd.read_csv('/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/visit/dfs_samples_2b.csv',
                      index_col='ID')
df_v_3b = pd.read_csv('/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/visit/dfs_samples_3b.csv',
                      index_col='ID')

In [85]:
df_indep = df_survey.join(df_quest, on='ID')
df_indep.head()

,Płeć,Rok_urodzenia,Kierunek_studiów,Rok_studiów,Specjalność,Wady_wzroku,Ilość_snu,Samopocz,Stres,Zmęcz,...,MAT,POL,HIST,Typ_uczenia,Quest1_before,Quest2_before,Quest1_after,Quest2_after,is_change_1,is_change_2
ID,,,,,,,,,,,,,,,,,,,,,
1,M,2002,MISMaP Geografia,2,SPOŁ-EKO,0,6,wysoki,niski,średni,...,VAL,VAL,NO_VAL,Kinestetyk,B,wykres,B,wykres,False,False
2,M,2003,Gospodarka przestrzenna,1,NO_VAL,1,<6,średni,niski,niski,...,VAL,VAL,NO_VAL,Brak dominującego,B,wykres,B,wykres,False,False
3,M,2001,Geografia,3,GEOINF,0,8,wysoki,średni,niski,...,VAL,NO_VAL,NO_VAL,Wzrokowiec,B,mapa,B,mapa,False,False
4,K,2000,Geografia,3,GEOINF,0,8,wysoki,niski,wysoki,...,NO_VAL,VAL,NO_VAL,Brak dominującego,B,mapa,B,wykres,False,True
5,K,2002,Geografia,3,GEOINF,0,7,wysoki,niski,niski,...,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec,B,tabela,B,tabela,False,False


In [86]:
# utworzenie zbioru dla zmiennych zależnych - grafiki A
# fiksacje
# całkowita długość
df_A = df_f_1a.filter(regex='Total').sum(axis=1).rename('Total_fix_withCom_1').reset_index()
df_A = df_A.join(df_f_2a.filter(regex='Total').sum(axis=1).rename('Total_fix_withCom_2'), on='ID')
df_A = df_A.join(df_f_3a.filter(regex='Total').sum(axis=1).rename('Total_fix_withCom_3'), on='ID')
df_A['Sum_TotalFix_WithCom'] = df_A[['Total_fix_withCom_1', 'Total_fix_withCom_2', 'Total_fix_withCom_3']].sum(axis=1)
df_A['Avg_TotalFix_WithCom'] = df_A[['Total_fix_withCom_1', 'Total_fix_withCom_2', 'Total_fix_withCom_3']].mean(axis=1)

df_A = df_A.join(df_f_1a.filter(regex='^Total_(?!com)').sum(axis=1).rename('Total_fix_withoutCom_1'), on='ID')
df_A = df_A.join(df_f_2a.filter(regex='^Total_(?!com)').sum(axis=1).rename('Total_fix_withoutCom_2'), on='ID')
df_A = df_A.join(df_f_3a.filter(regex='^Total_(?!com)').sum(axis=1).rename('Total_fix_withoutCom_3'), on='ID')
df_A['Sum_TotalFix_WithoutCom'] = df_A[['Total_fix_withoutCom_1', 'Total_fix_withoutCom_2', 'Total_fix_withoutCom_3']].sum(axis=1)
df_A['Avg_TotalFix_WithoutCom'] = df_A[['Total_fix_withoutCom_1', 'Total_fix_withoutCom_2', 'Total_fix_withoutCom_3']].mean(axis=1)

df_A = df_A.join(df_f_1a.filter(regex='Total_com').sum(axis=1).rename('Total_fix_onlyCom_1'), on='ID')
df_A = df_A.join(df_f_2a.filter(regex='Total_com').sum(axis=1).rename('Total_fix_onlyCom_2'), on='ID')
df_A = df_A.join(df_f_3a.filter(regex='Total_com').sum(axis=1).rename('Total_fix_onlyCom_3'), on='ID')
df_A['Sum_TotalFix_OnlyCom'] = df_A[['Total_fix_onlyCom_1', 'Total_fix_onlyCom_2', 'Total_fix_onlyCom_3']].sum(axis=1)
df_A['Avg_TotalFix_OnlyCom'] = df_A[['Total_fix_onlyCom_1', 'Total_fix_onlyCom_2', 'Total_fix_onlyCom_3']].mean(axis=1)

# średnia długość
df_A = df_A.join(df_f_1a.filter(regex='^Avg_(?!pupil)').sum(axis=1).rename('Avg_fix_withCom_1'), on='ID')
df_A = df_A.join(df_f_2a.filter(regex='^Avg_(?!pupil)').sum(axis=1).rename('Avg_fix_withCom_2'), on='ID')
df_A = df_A.join(df_f_3a.filter(regex='^Avg_(?!pupil)').sum(axis=1).rename('Avg_fix_withCom_3'), on='ID')
df_A['Sum_AvgFix_WithCom'] = df_A[['Avg_fix_withCom_1', 'Avg_fix_withCom_2', 'Avg_fix_withCom_3']].sum(axis=1)
df_A['Avg_AvgFix_WithCom'] = df_A[['Avg_fix_withCom_1', 'Avg_fix_withCom_2', 'Avg_fix_withCom_3']].mean(axis=1)

df_A = df_A.join(df_f_1a.filter(regex='^Avg_(?!com|pupil_.*$)').sum(axis=1).rename('Avg_fix_withoutCom_1'), on='ID')
df_A = df_A.join(df_f_2a.filter(regex='^Avg_(?!com|pupil_.*$)').sum(axis=1).rename('Avg_fix_withoutCom_2'), on='ID')
df_A = df_A.join(df_f_3a.filter(regex='^Avg_(?!com|pupil_.*$)').sum(axis=1).rename('Avg_fix_withoutCom_3'), on='ID')
df_A['Sum_AvgFix_WithoutCom'] = df_A[['Avg_fix_withoutCom_1', 'Avg_fix_withoutCom_2', 'Avg_fix_withoutCom_3']].sum(axis=1)
df_A['Avg_AvgFix_WithoutCom'] = df_A[['Avg_fix_withoutCom_1', 'Avg_fix_withoutCom_2', 'Avg_fix_withoutCom_3']].mean(axis=1)

df_A = df_A.join(df_f_1a.filter(regex='Avg_com').sum(axis=1).rename('Avg_fix_onlyCom_1'), on='ID')
df_A = df_A.join(df_f_2a.filter(regex='Avg_com').sum(axis=1).rename('Avg_fix_onlyCom_2'), on='ID')
df_A = df_A.join(df_f_3a.filter(regex='Avg_com').sum(axis=1).rename('Avg_fix_onlyCom_3'), on='ID')
df_A['Sum_AvgFix_OnlyCom'] = df_A[['Avg_fix_onlyCom_1', 'Avg_fix_onlyCom_2', 'Avg_fix_onlyCom_3']].sum(axis=1)
df_A['Avg_AvgFix_OnlyCom'] = df_A[['Avg_fix_onlyCom_1', 'Avg_fix_onlyCom_2', 'Avg_fix_onlyCom_3']].mean(axis=1)

# liczba
df_A = df_A.join(df_f_1a.filter(regex='Num').sum(axis=1).rename('Num_fix_withCom_1'), on='ID')
df_A = df_A.join(df_f_2a.filter(regex='Num').sum(axis=1).rename('Num_fix_withCom_2'), on='ID')
df_A = df_A.join(df_f_3a.filter(regex='Num').sum(axis=1).rename('Num_fix_withCom_3'), on='ID')
df_A['Sum_NumFix_WithCom'] = df_A[['Num_fix_withCom_1', 'Num_fix_withCom_2', 'Num_fix_withCom_3']].sum(axis=1)
df_A['Avg_NumFix_WithCom'] = df_A[['Num_fix_withCom_1', 'Num_fix_withCom_2', 'Num_fix_withCom_3']].mean(axis=1)

df_A = df_A.join(df_f_1a.filter(regex='^Num_(?!com)').sum(axis=1).rename('Num_fix_withoutCom_1'), on='ID')
df_A = df_A.join(df_f_2a.filter(regex='^Num_(?!com)').sum(axis=1).rename('Num_fix_withoutCom_2'), on='ID')
df_A = df_A.join(df_f_3a.filter(regex='^Num_(?!com)').sum(axis=1).rename('Num_fix_withoutCom_3'), on='ID')
df_A['Sum_NumFix_WithoutCom'] = df_A[['Num_fix_withoutCom_1', 'Num_fix_withoutCom_2', 'Num_fix_withoutCom_3']].sum(axis=1)
df_A['Avg_NumFix_WithoutCom'] = df_A[['Num_fix_withoutCom_1', 'Num_fix_withoutCom_2', 'Num_fix_withoutCom_3']].mean(axis=1)

df_A = df_A.join(df_f_1a.filter(regex='Num_com').sum(axis=1).rename('Num_fix_onlyCom_1'), on='ID')
df_A = df_A.join(df_f_2a.filter(regex='Num_com').sum(axis=1).rename('Num_fix_onlyCom_2'), on='ID')
df_A = df_A.join(df_f_3a.filter(regex='Num_com').sum(axis=1).rename('Num_fix_onlyCom_3'), on='ID')
df_A['Sum_NumFix_OnlyCom'] = df_A[['Num_fix_onlyCom_1', 'Num_fix_onlyCom_2', 'Num_fix_onlyCom_3']].sum(axis=1)
df_A['Avg_NumFix_OnlyCom'] = df_A[['Num_fix_onlyCom_1', 'Num_fix_onlyCom_2', 'Num_fix_onlyCom_3']].mean(axis=1)

# średnia wielkość źrenicy
df_A = df_A.join(df_f_1a.filter(regex='Avg_pupil').mean(axis=1).rename('Avg_pupil_withCom_1'), on='ID')
df_A = df_A.join(df_f_2a.filter(regex='Avg_pupil').mean(axis=1).rename('Avg_pupil_withCom_2'), on='ID')
df_A = df_A.join(df_f_3a.filter(regex='Avg_pupil').mean(axis=1).rename('Avg_pupil_withCom_3'), on='ID')
df_A['Avg_AvgPupil_WithCom'] = df_A[['Avg_pupil_withCom_1', 'Avg_pupil_withCom_2', 'Avg_pupil_withCom_3']].mean(axis=1)

df_A = df_A.join(df_f_1a.filter(regex='^Avg_pupil_(?!com$)').mean(axis=1).rename('Avg_pupil_withoutCom_1'), on='ID')
df_A = df_A.join(df_f_2a.filter(regex='^Avg_pupil_(?!com$)').mean(axis=1).rename('Avg_pupil_withoutCom_2'), on='ID')
df_A = df_A.join(df_f_3a.filter(regex='^Avg_pupil_(?!com$)').mean(axis=1).rename('Avg_pupil_withoutCom_3'), on='ID')
df_A['Avg_AvgPupil_WithoutCom'] = df_A[['Avg_pupil_withoutCom_1', 'Avg_pupil_withoutCom_2', 'Avg_pupil_withoutCom_3']].mean(axis=1)

df_A = df_A.join(df_f_1a.filter(regex='Avg_pupil_com').mean(axis=1).rename('Avg_pupil_onlyCom_1'), on='ID')
df_A = df_A.join(df_f_2a.filter(regex='Avg_pupil_com').mean(axis=1).rename('Avg_pupil_onlyCom_2'), on='ID')
df_A = df_A.join(df_f_3a.filter(regex='Avg_pupil_com').mean(axis=1).rename('Avg_pupil_onlyCom_3'), on='ID')
df_A['Avg_AvgPupil_OnlyCom'] = df_A[['Avg_pupil_onlyCom_1', 'Avg_pupil_onlyCom_2', 'Avg_pupil_onlyCom_3']].mean(axis=1)

# wizyty
# całkowita długość
df_A = df_A.join(df_v_1a.filter(regex='Total').sum(axis=1).rename('Total_visit_withCom_1'), on='ID')
df_A = df_A.join(df_v_2a.filter(regex='Total').sum(axis=1).rename('Total_visit_withCom_2'), on='ID')
df_A = df_A.join(df_v_3a.filter(regex='Total').sum(axis=1).rename('Total_visit_withCom_3'), on='ID')
df_A['Sum_TotalVisit_WithCom'] = df_A[['Total_visit_withCom_1', 'Total_visit_withCom_2', 'Total_visit_withCom_3']].sum(
    axis=1)
df_A['Avg_TotalVisit_WithCom'] = df_A[['Total_visit_withCom_1', 'Total_visit_withCom_2', 'Total_visit_withCom_3']].mean(
    axis=1)

df_A = df_A.join(df_v_1a.filter(regex='^Total_(?!com)').sum(axis=1).rename('Total_visit_withoutCom_1'), on='ID')
df_A = df_A.join(df_v_2a.filter(regex='^Total_(?!com)').sum(axis=1).rename('Total_visit_withoutCom_2'), on='ID')
df_A = df_A.join(df_v_3a.filter(regex='^Total_(?!com)').sum(axis=1).rename('Total_visit_withoutCom_3'), on='ID')
df_A['Sum_TotalVisit_WithoutCom'] = df_A[['Total_visit_withoutCom_1', 'Total_visit_withoutCom_2', 'Total_visit_withoutCom_3']].sum(axis=1)
df_A['Avg_TotalVisit_WithoutCom'] = df_A[['Total_visit_withoutCom_1', 'Total_visit_withoutCom_2', 'Total_visit_withoutCom_3']].mean(axis=1)

df_A = df_A.join(df_v_1a.filter(regex='Total_com').sum(axis=1).rename('Total_visit_onlyCom_1'), on='ID')
df_A = df_A.join(df_v_2a.filter(regex='Total_com').sum(axis=1).rename('Total_visit_onlyCom_2'), on='ID')
df_A = df_A.join(df_v_3a.filter(regex='Total_com').sum(axis=1).rename('Total_visit_onlyCom_3'), on='ID')
df_A['Sum_TotalVisit_OnlyCom'] = df_A[['Total_visit_onlyCom_1', 'Total_visit_onlyCom_2', 'Total_visit_onlyCom_3']].sum(
    axis=1)
df_A['Avg_TotalVisit_OnlyCom'] = df_A[['Total_visit_onlyCom_1', 'Total_visit_onlyCom_2', 'Total_visit_onlyCom_3']].mean(
    axis=1)

# średnia długość
df_A = df_A.join(df_v_1a.filter(regex='^Avg_(?!pupil)').sum(axis=1).rename('Avg_visit_withCom_1'), on='ID')
df_A = df_A.join(df_v_2a.filter(regex='^Avg_(?!pupil)').sum(axis=1).rename('Avg_visit_withCom_2'), on='ID')
df_A = df_A.join(df_v_3a.filter(regex='^Avg_(?!pupil)').sum(axis=1).rename('Avg_visit_withCom_3'), on='ID')
df_A['Sum_AvgVisit_WithCom'] = df_A[['Avg_visit_withCom_1', 'Avg_visit_withCom_2', 'Avg_visit_withCom_3']].sum(axis=1)
df_A['Avg_AvgVisit_WithCom'] = df_A[['Avg_visit_withCom_1', 'Avg_visit_withCom_2', 'Avg_visit_withCom_3']].mean(axis=1)

df_A = df_A.join(df_v_1a.filter(regex='^Avg_(?!com|pupil_.*$)').sum(axis=1).rename('Avg_visit_withoutCom_1'), on='ID')
df_A = df_A.join(df_v_2a.filter(regex='^Avg_(?!com|pupil_.*$)').sum(axis=1).rename('Avg_visit_withoutCom_2'), on='ID')
df_A = df_A.join(df_v_3a.filter(regex='^Avg_(?!com|pupil_.*$)').sum(axis=1).rename('Avg_visit_withoutCom_3'), on='ID')
df_A['Sum_AvgVisit_WithoutCom'] = df_A[['Avg_visit_withoutCom_1', 'Avg_visit_withoutCom_2', 'Avg_visit_withoutCom_3']].sum(axis=1)
df_A['Avg_AvgVisit_WithoutCom'] = df_A[['Avg_visit_withoutCom_1', 'Avg_visit_withoutCom_2', 'Avg_visit_withoutCom_3']].mean(axis=1)

df_A = df_A.join(df_v_1a.filter(regex='Avg_com').sum(axis=1).rename('Avg_visit_onlyCom_1'), on='ID')
df_A = df_A.join(df_v_2a.filter(regex='Avg_com').sum(axis=1).rename('Avg_visit_onlyCom_2'), on='ID')
df_A = df_A.join(df_v_3a.filter(regex='Avg_com').sum(axis=1).rename('Avg_visit_onlyCom_3'), on='ID')
df_A['Sum_AvgVisit_OnlyCom'] = df_A[['Avg_visit_onlyCom_1', 'Avg_visit_onlyCom_2', 'Avg_visit_onlyCom_3']].sum(axis=1)
df_A['Avg_AvgVisit_OnlyCom'] = df_A[['Avg_visit_onlyCom_1', 'Avg_visit_onlyCom_2', 'Avg_visit_onlyCom_3']].mean(axis=1)

# liczba
df_A = df_A.join(df_v_1a.filter(regex='Num').sum(axis=1).rename('Num_visit_withCom_1'), on='ID')
df_A = df_A.join(df_v_2a.filter(regex='Num').sum(axis=1).rename('Num_visit_withCom_2'), on='ID')
df_A = df_A.join(df_v_3a.filter(regex='Num').sum(axis=1).rename('Num_visit_withCom_3'), on='ID')
df_A['Sum_NumVisit_WithCom'] = df_A[['Num_visit_withCom_1', 'Num_visit_withCom_2', 'Num_visit_withCom_3']].sum(axis=1)
df_A['Avg_NumVisit_WithCom'] = df_A[['Num_visit_withCom_1', 'Num_visit_withCom_2', 'Num_visit_withCom_3']].mean(axis=1)

df_A = df_A.join(df_v_1a.filter(regex='^Num_(?!com)').sum(axis=1).rename('Num_visit_withoutCom_1'), on='ID')
df_A = df_A.join(df_v_2a.filter(regex='^Num_(?!com)').sum(axis=1).rename('Num_visit_withoutCom_2'), on='ID')
df_A = df_A.join(df_v_3a.filter(regex='^Num_(?!com)').sum(axis=1).rename('Num_visit_withoutCom_3'), on='ID')
df_A['Sum_NumVisit_WithoutCom'] = df_A[['Num_visit_withoutCom_1', 'Num_visit_withoutCom_2', 'Num_visit_withoutCom_3']].sum(axis=1)
df_A['Avg_NumVisit_WithoutCom'] = df_A[['Num_visit_withoutCom_1', 'Num_visit_withoutCom_2', 'Num_visit_withoutCom_3']].mean(axis=1)

df_A = df_A.join(df_v_1a.filter(regex='Num_com').sum(axis=1).rename('Num_visit_onlyCom_1'), on='ID')
df_A = df_A.join(df_v_2a.filter(regex='Num_com').sum(axis=1).rename('Num_visit_onlyCom_2'), on='ID')
df_A = df_A.join(df_v_3a.filter(regex='Num_com').sum(axis=1).rename('Num_visit_onlyCom_3'), on='ID')
df_A['Sum_NumVisit_OnlyCom'] = df_A[['Num_visit_onlyCom_1', 'Num_visit_onlyCom_2', 'Num_visit_onlyCom_3']].sum(axis=1)
df_A['Avg_NumVisit_OnlyCom'] = df_A[['Num_visit_onlyCom_1', 'Num_visit_onlyCom_2', 'Num_visit_onlyCom_3']].mean(axis=1)

# ------------------------------------------
# poprawność
df_A = df_A.join(df_answers['SumCorr_a'].rename('Sum_Corr'), on='ID')
df_A = df_A.join(df_answers['1a_corr'].rename('Corr_1'), on='ID')
df_A = df_A.join(df_answers['2a_corr'].rename('Corr_2'), on='ID')
df_A = df_A.join(df_answers['3a_corr'].rename('Corr_3'), on='ID')
df_A['Avg_Corr'] = df_A['Sum_Corr'] / 3

# trudność
df_A = df_A.join(df_answers['SumTrud_a'].rename('Sum_Diff'), on='ID')
df_A = df_A.join(df_answers['1a_trud'].rename('Diff_1'), on='ID')
df_A = df_A.join(df_answers['2a_trud'].rename('Diff_2'), on='ID')
df_A = df_A.join(df_answers['3a_trud'].rename('Diff_3'), on='ID')
df_A['Avg_Diff'] = df_A['Sum_Diff'] / 3

# czas odpowiedzi
df_A = df_A.join(df_time['SumTime_a'].rename('Sum_Time'), on='ID')
df_A = df_A.join(df_time['Duration_1a'].rename('Time_1'), on='ID')
df_A = df_A.join(df_time['Duration_2a'].rename('Time_2'), on='ID')
df_A = df_A.join(df_time['Duration_3a'].rename('Time_3'), on='ID')
df_A['Avg_Time'] = df_A['Sum_Time'] / 3

# grupa
df_A['Group'] = 'A'

df_A['idx'] = range(0, 40)
df_A.set_index('idx', inplace=True)
df_A

,ID,Total_fix_withCom_1,Total_fix_withCom_2,Total_fix_withCom_3,Sum_TotalFix_WithCom,Avg_TotalFix_WithCom,Total_fix_withoutCom_1,Total_fix_withoutCom_2,Total_fix_withoutCom_3,Sum_TotalFix_WithoutCom,...,Diff_1,Diff_2,Diff_3,Avg_Diff,Sum_Time,Time_1,Time_2,Time_3,Avg_Time,Group
idx,,,,,,,,,,,,,,,,,,,,,
0,46,4799,10809,9274,24882,8294.000000,3133,4576,7682,15391,...,1,2,2,1.666667,141444,34682,38765,67997,47148.000000,A
1,37,36009,21199,37874,95082,31694.000000,36009,20967,36079,93055,...,2,2,4,2.666667,137677,53564,35032,49081,45892.333333,A
2,23,15724,2400,5808,23932,7977.333333,13774,842,2750,17366,...,4,1,1,2.000000,119128,85546,12266,21316,39709.333333,A
3,40,43426,7501,13058,63985,21328.333333,42868,6718,9741,59327,...,4,3,2,3.000000,92095,61197,10999,19899,30698.333333,A
4,19,35214,9624,37404,82242,27414.000000,33214,7183,33022,73419,...,3,1,2,2.000000,99761,42031,12816,44914,33253.666667,A
5,31,34481,26914,29965,91360,30453.333333,26340,18965,15083,60388,...,2,3,2,2.333333,112294,41881,32832,37581,37431.333333,A
6,38,32123,2442,24498,59063,19687.666667,31338,1767,24498,57603,...,4,1,3,2.666667,114461,57031,8416,49014,38153.666667,A
7,35,82120,8255,14701,105076,35025.333333,79503,6547,12410,98460,...,5,2,4,3.666667,149809,117478,11699,20632,49936.333333,A
8,42,41152,11949,28752,81853,27284.333333,37827,9432,23268,70527,...,2,2,2,2.000000,126727,61864,22732,42131,42242.333333,A


In [87]:
# utworzenie zbioru dla zmiennych zależnych - grafiki B
# fiksacje
# całkowita długość
df_B = df_f_1b.filter(regex='Total').sum(axis=1).rename('Total_fix_withCom_1').reset_index()
df_B = df_B.join(df_f_2b.filter(regex='Total').sum(axis=1).rename('Total_fix_withCom_2'), on='ID')
df_B = df_B.join(df_f_3b.filter(regex='Total').sum(axis=1).rename('Total_fix_withCom_3'), on='ID')
df_B['Sum_TotalFix_WithCom'] = df_B[['Total_fix_withCom_1', 'Total_fix_withCom_2', 'Total_fix_withCom_3']].sum(axis=1)
df_B['Avg_TotalFix_WithCom'] = df_B[['Total_fix_withCom_1', 'Total_fix_withCom_2', 'Total_fix_withCom_3']].mean(axis=1)

df_B = df_B.join(df_f_1b.filter(regex='^Total_(?!com)').sum(axis=1).rename('Total_fix_withoutCom_1'), on='ID')
df_B = df_B.join(df_f_2b.filter(regex='^Total_(?!com)').sum(axis=1).rename('Total_fix_withoutCom_2'), on='ID')
df_B = df_B.join(df_f_3b.filter(regex='^Total_(?!com)').sum(axis=1).rename('Total_fix_withoutCom_3'), on='ID')
df_B['Sum_TotalFix_WithoutCom'] = df_B[['Total_fix_withoutCom_1', 'Total_fix_withoutCom_2', 'Total_fix_withoutCom_3']].sum(axis=1)
df_B['Avg_TotalFix_WithoutCom'] = df_B[['Total_fix_withoutCom_1', 'Total_fix_withoutCom_2', 'Total_fix_withoutCom_3']].mean(axis=1)

df_B = df_B.join(df_f_1b.filter(regex='Total_com').sum(axis=1).rename('Total_fix_onlyCom_1'), on='ID')
df_B = df_B.join(df_f_2b.filter(regex='Total_com').sum(axis=1).rename('Total_fix_onlyCom_2'), on='ID')
df_B = df_B.join(df_f_3b.filter(regex='Total_com').sum(axis=1).rename('Total_fix_onlyCom_3'), on='ID')
df_B['Sum_TotalFix_OnlyCom'] = df_B[['Total_fix_onlyCom_1', 'Total_fix_onlyCom_2', 'Total_fix_onlyCom_3']].sum(axis=1)
df_B['Avg_TotalFix_OnlyCom'] = df_B[['Total_fix_onlyCom_1', 'Total_fix_onlyCom_2', 'Total_fix_onlyCom_3']].mean(axis=1)

# średnia długość
df_B = df_B.join(df_f_1b.filter(regex='^Avg_(?!pupil)').sum(axis=1).rename('Avg_fix_withCom_1'), on='ID')
df_B = df_B.join(df_f_2b.filter(regex='^Avg_(?!pupil)').sum(axis=1).rename('Avg_fix_withCom_2'), on='ID')
df_B = df_B.join(df_f_3b.filter(regex='^Avg_(?!pupil)').sum(axis=1).rename('Avg_fix_withCom_3'), on='ID')
df_B['Sum_AvgFix_WithCom'] = df_B[['Avg_fix_withCom_1', 'Avg_fix_withCom_2', 'Avg_fix_withCom_3']].sum(axis=1)
df_B['Avg_AvgFix_WithCom'] = df_B[['Avg_fix_withCom_1', 'Avg_fix_withCom_2', 'Avg_fix_withCom_3']].mean(axis=1)

df_B = df_B.join(df_f_1b.filter(regex='^Avg_(?!com|pupil_.*$)').sum(axis=1).rename('Avg_fix_withoutCom_1'), on='ID')
df_B = df_B.join(df_f_2b.filter(regex='^Avg_(?!com|pupil_.*$)').sum(axis=1).rename('Avg_fix_withoutCom_2'), on='ID')
df_B = df_B.join(df_f_3b.filter(regex='^Avg_(?!com|pupil_.*$)').sum(axis=1).rename('Avg_fix_withoutCom_3'), on='ID')
df_B['Sum_AvgFix_WithoutCom'] = df_B[['Avg_fix_withoutCom_1', 'Avg_fix_withoutCom_2', 'Avg_fix_withoutCom_3']].sum(axis=1)
df_B['Avg_AvgFix_WithoutCom'] = df_B[['Avg_fix_withoutCom_1', 'Avg_fix_withoutCom_2', 'Avg_fix_withoutCom_3']].mean(axis=1)

df_B = df_B.join(df_f_1b.filter(regex='Avg_com').sum(axis=1).rename('Avg_fix_onlyCom_1'), on='ID')
df_B = df_B.join(df_f_2b.filter(regex='Avg_com').sum(axis=1).rename('Avg_fix_onlyCom_2'), on='ID')
df_B = df_B.join(df_f_3b.filter(regex='Avg_com').sum(axis=1).rename('Avg_fix_onlyCom_3'), on='ID')
df_B['Sum_AvgFix_OnlyCom'] = df_B[['Avg_fix_onlyCom_1', 'Avg_fix_onlyCom_2', 'Avg_fix_onlyCom_3']].sum(axis=1)
df_B['Avg_AvgFix_OnlyCom'] = df_B[['Avg_fix_onlyCom_1', 'Avg_fix_onlyCom_2', 'Avg_fix_onlyCom_3']].mean(axis=1)

# liczba
df_B = df_B.join(df_f_1b.filter(regex='Num').sum(axis=1).rename('Num_fix_withCom_1'), on='ID')
df_B = df_B.join(df_f_2b.filter(regex='Num').sum(axis=1).rename('Num_fix_withCom_2'), on='ID')
df_B = df_B.join(df_f_3b.filter(regex='Num').sum(axis=1).rename('Num_fix_withCom_3'), on='ID')
df_B['Sum_NumFix_WithCom'] = df_B[['Num_fix_withCom_1', 'Num_fix_withCom_2', 'Num_fix_withCom_3']].sum(axis=1)
df_B['Avg_NumFix_WithCom'] = df_B[['Num_fix_withCom_1', 'Num_fix_withCom_2', 'Num_fix_withCom_3']].mean(axis=1)

df_B = df_B.join(df_f_1b.filter(regex='^Num_(?!com)').sum(axis=1).rename('Num_fix_withoutCom_1'), on='ID')
df_B = df_B.join(df_f_2b.filter(regex='^Num_(?!com)').sum(axis=1).rename('Num_fix_withoutCom_2'), on='ID')
df_B = df_B.join(df_f_3b.filter(regex='^Num_(?!com)').sum(axis=1).rename('Num_fix_withoutCom_3'), on='ID')
df_B['Sum_NumFix_WithoutCom'] = df_B[['Num_fix_withoutCom_1', 'Num_fix_withoutCom_2', 'Num_fix_withoutCom_3']].sum(axis=1)
df_B['Avg_NumFix_WithoutCom'] = df_B[['Num_fix_withoutCom_1', 'Num_fix_withoutCom_2', 'Num_fix_withoutCom_3']].mean(axis=1)

df_B = df_B.join(df_f_1b.filter(regex='Num_com').sum(axis=1).rename('Num_fix_onlyCom_1'), on='ID')
df_B = df_B.join(df_f_2b.filter(regex='Num_com').sum(axis=1).rename('Num_fix_onlyCom_2'), on='ID')
df_B = df_B.join(df_f_3b.filter(regex='Num_com').sum(axis=1).rename('Num_fix_onlyCom_3'), on='ID')
df_B['Sum_NumFix_OnlyCom'] = df_B[['Num_fix_onlyCom_1', 'Num_fix_onlyCom_2', 'Num_fix_onlyCom_3']].sum(axis=1)
df_B['Avg_NumFix_OnlyCom'] = df_B[['Num_fix_onlyCom_1', 'Num_fix_onlyCom_2', 'Num_fix_onlyCom_3']].mean(axis=1)

# średnia wielkość źrenicy
df_B = df_B.join(df_f_1b.filter(regex='Avg_pupil').mean(axis=1).rename('Avg_pupil_withCom_1'), on='ID')
df_B = df_B.join(df_f_2b.filter(regex='Avg_pupil').mean(axis=1).rename('Avg_pupil_withCom_2'), on='ID')
df_B = df_B.join(df_f_3b.filter(regex='Avg_pupil').mean(axis=1).rename('Avg_pupil_withCom_3'), on='ID')
df_B['Avg_AvgPupil_WithCom'] = df_B[['Avg_pupil_withCom_1', 'Avg_pupil_withCom_2', 'Avg_pupil_withCom_3']].mean(axis=1)

df_B = df_B.join(df_f_1b.filter(regex='^Avg_pupil_(?!com$)').mean(axis=1).rename('Avg_pupil_withoutCom_1'), on='ID')
df_B = df_B.join(df_f_2b.filter(regex='^Avg_pupil_(?!com$)').mean(axis=1).rename('Avg_pupil_withoutCom_2'), on='ID')
df_B = df_B.join(df_f_3b.filter(regex='^Avg_pupil_(?!com$)').mean(axis=1).rename('Avg_pupil_withoutCom_3'), on='ID')
df_B['Avg_AvgPupil_WithoutCom'] = df_B[['Avg_pupil_withoutCom_1', 'Avg_pupil_withoutCom_2', 'Avg_pupil_withoutCom_3']].mean(axis=1)

df_B = df_B.join(df_f_1b.filter(regex='Avg_pupil_com').mean(axis=1).rename('Avg_pupil_onlyCom_1'), on='ID')
df_B = df_B.join(df_f_2b.filter(regex='Avg_pupil_com').mean(axis=1).rename('Avg_pupil_onlyCom_2'), on='ID')
df_B = df_B.join(df_f_3b.filter(regex='Avg_pupil_com').mean(axis=1).rename('Avg_pupil_onlyCom_3'), on='ID')
df_B['Avg_AvgPupil_OnlyCom'] = df_B[['Avg_pupil_onlyCom_1', 'Avg_pupil_onlyCom_2', 'Avg_pupil_onlyCom_3']].mean(axis=1)

# wizyty
# całkowita długość
df_B = df_B.join(df_v_1b.filter(regex='Total').sum(axis=1).rename('Total_visit_withCom_1'), on='ID')
df_B = df_B.join(df_v_2b.filter(regex='Total').sum(axis=1).rename('Total_visit_withCom_2'), on='ID')
df_B = df_B.join(df_v_3b.filter(regex='Total').sum(axis=1).rename('Total_visit_withCom_3'), on='ID')
df_B['Sum_TotalVisit_WithCom'] = df_B[['Total_visit_withCom_1', 'Total_visit_withCom_2', 'Total_visit_withCom_3']].sum(
    axis=1)
df_B['Avg_TotalVisit_WithCom'] = df_B[['Total_visit_withCom_1', 'Total_visit_withCom_2', 'Total_visit_withCom_3']].mean(
    axis=1)

df_B = df_B.join(df_v_1b.filter(regex='^Total_(?!com)').sum(axis=1).rename('Total_visit_withoutCom_1'), on='ID')
df_B = df_B.join(df_v_2b.filter(regex='^Total_(?!com)').sum(axis=1).rename('Total_visit_withoutCom_2'), on='ID')
df_B = df_B.join(df_v_3b.filter(regex='^Total_(?!com)').sum(axis=1).rename('Total_visit_withoutCom_3'), on='ID')
df_B['Sum_TotalVisit_WithoutCom'] = df_B[['Total_visit_withoutCom_1', 'Total_visit_withoutCom_2', 'Total_visit_withoutCom_3']].sum(axis=1)
df_B['Avg_TotalVisit_WithoutCom'] = df_B[['Total_visit_withoutCom_1', 'Total_visit_withoutCom_2', 'Total_visit_withoutCom_3']].mean(axis=1)

df_B = df_B.join(df_v_1b.filter(regex='Total_com').sum(axis=1).rename('Total_visit_onlyCom_1'), on='ID')
df_B = df_B.join(df_v_2b.filter(regex='Total_com').sum(axis=1).rename('Total_visit_onlyCom_2'), on='ID')
df_B = df_B.join(df_v_3b.filter(regex='Total_com').sum(axis=1).rename('Total_visit_onlyCom_3'), on='ID')
df_B['Sum_TotalVisit_OnlyCom'] = df_B[['Total_visit_onlyCom_1', 'Total_visit_onlyCom_2', 'Total_visit_onlyCom_3']].sum(
    axis=1)
df_B['Avg_TotalVisit_OnlyCom'] = df_B[['Total_visit_onlyCom_1', 'Total_visit_onlyCom_2', 'Total_visit_onlyCom_3']].mean(
    axis=1)

# średnia długość
df_B = df_B.join(df_v_1b.filter(regex='^Avg_(?!pupil)').sum(axis=1).rename('Avg_visit_withCom_1'), on='ID')
df_B = df_B.join(df_v_2b.filter(regex='^Avg_(?!pupil)').sum(axis=1).rename('Avg_visit_withCom_2'), on='ID')
df_B = df_B.join(df_v_3b.filter(regex='^Avg_(?!pupil)').sum(axis=1).rename('Avg_visit_withCom_3'), on='ID')
df_B['Sum_AvgVisit_WithCom'] = df_B[['Avg_visit_withCom_1', 'Avg_visit_withCom_2', 'Avg_visit_withCom_3']].sum(axis=1)
df_B['Avg_AvgVisit_WithCom'] = df_B[['Avg_visit_withCom_1', 'Avg_visit_withCom_2', 'Avg_visit_withCom_3']].mean(axis=1)

df_B = df_B.join(df_v_1b.filter(regex='^Avg_(?!com|pupil_.*$)').sum(axis=1).rename('Avg_visit_withoutCom_1'), on='ID')
df_B = df_B.join(df_v_2b.filter(regex='^Avg_(?!com|pupil_.*$)').sum(axis=1).rename('Avg_visit_withoutCom_2'), on='ID')
df_B = df_B.join(df_v_3b.filter(regex='^Avg_(?!com|pupil_.*$)').sum(axis=1).rename('Avg_visit_withoutCom_3'), on='ID')
df_B['Sum_AvgVisit_WithoutCom'] = df_B[['Avg_visit_withoutCom_1', 'Avg_visit_withoutCom_2', 'Avg_visit_withoutCom_3']].sum(axis=1)
df_B['Avg_AvgVisit_WithoutCom'] = df_B[['Avg_visit_withoutCom_1', 'Avg_visit_withoutCom_2', 'Avg_visit_withoutCom_3']].mean(axis=1)

df_B = df_B.join(df_v_1b.filter(regex='Avg_com').sum(axis=1).rename('Avg_visit_onlyCom_1'), on='ID')
df_B = df_B.join(df_v_2b.filter(regex='Avg_com').sum(axis=1).rename('Avg_visit_onlyCom_2'), on='ID')
df_B = df_B.join(df_v_3b.filter(regex='Avg_com').sum(axis=1).rename('Avg_visit_onlyCom_3'), on='ID')
df_B['Sum_AvgVisit_OnlyCom'] = df_B[['Avg_visit_onlyCom_1', 'Avg_visit_onlyCom_2', 'Avg_visit_onlyCom_3']].sum(axis=1)
df_B['Avg_AvgVisit_OnlyCom'] = df_B[['Avg_visit_onlyCom_1', 'Avg_visit_onlyCom_2', 'Avg_visit_onlyCom_3']].mean(axis=1)

# liczba
df_B = df_B.join(df_v_1b.filter(regex='Num').sum(axis=1).rename('Num_visit_withCom_1'), on='ID')
df_B = df_B.join(df_v_2b.filter(regex='Num').sum(axis=1).rename('Num_visit_withCom_2'), on='ID')
df_B = df_B.join(df_v_3b.filter(regex='Num').sum(axis=1).rename('Num_visit_withCom_3'), on='ID')
df_B['Sum_NumVisit_WithCom'] = df_B[['Num_visit_withCom_1', 'Num_visit_withCom_2', 'Num_visit_withCom_3']].sum(axis=1)
df_B['Avg_NumVisit_WithCom'] = df_B[['Num_visit_withCom_1', 'Num_visit_withCom_2', 'Num_visit_withCom_3']].mean(axis=1)

df_B = df_B.join(df_v_1b.filter(regex='^Num_(?!com)').sum(axis=1).rename('Num_visit_withoutCom_1'), on='ID')
df_B = df_B.join(df_v_2b.filter(regex='^Num_(?!com)').sum(axis=1).rename('Num_visit_withoutCom_2'), on='ID')
df_B = df_B.join(df_v_3b.filter(regex='^Num_(?!com)').sum(axis=1).rename('Num_visit_withoutCom_3'), on='ID')
df_B['Sum_NumVisit_WithoutCom'] = df_B[['Num_visit_withoutCom_1', 'Num_visit_withoutCom_2', 'Num_visit_withoutCom_3']].sum(axis=1)
df_B['Avg_NumVisit_WithoutCom'] = df_B[['Num_visit_withoutCom_1', 'Num_visit_withoutCom_2', 'Num_visit_withoutCom_3']].mean(axis=1)

df_B = df_B.join(df_v_1b.filter(regex='Num_com').sum(axis=1).rename('Num_visit_onlyCom_1'), on='ID')
df_B = df_B.join(df_v_2b.filter(regex='Num_com').sum(axis=1).rename('Num_visit_onlyCom_2'), on='ID')
df_B = df_B.join(df_v_3b.filter(regex='Num_com').sum(axis=1).rename('Num_visit_onlyCom_3'), on='ID')
df_B['Sum_NumVisit_OnlyCom'] = df_B[['Num_visit_onlyCom_1', 'Num_visit_onlyCom_2', 'Num_visit_onlyCom_3']].sum(axis=1)
df_B['Avg_NumVisit_OnlyCom'] = df_B[['Num_visit_onlyCom_1', 'Num_visit_onlyCom_2', 'Num_visit_onlyCom_3']].mean(axis=1)

# ------------------------------------------
# poprawność
df_B = df_B.join(df_answers['SumCorr_b'].rename('Sum_Corr'), on='ID')
df_B = df_B.join(df_answers['1b_corr'].rename('Corr_1'), on='ID')
df_B = df_B.join(df_answers['2b_corr'].rename('Corr_2'), on='ID')
df_B = df_B.join(df_answers['3b_corr'].rename('Corr_3'), on='ID')
df_B['Avg_Corr'] = df_B['Sum_Corr'] / 3

# trudność
df_B = df_B.join(df_answers['SumTrud_b'].rename('Sum_Diff'), on='ID')
df_B = df_B.join(df_answers['1b_trud'].rename('Diff_1'), on='ID')
df_B = df_B.join(df_answers['2b_trud'].rename('Diff_2'), on='ID')
df_B = df_B.join(df_answers['3b_trud'].rename('Diff_3'), on='ID')
df_B['Avg_Diff'] = df_B['Sum_Diff'] / 3

# czas odpowiedzi
df_B = df_B.join(df_time['SumTime_b'].rename('Sum_Time'), on='ID')
df_B = df_B.join(df_time['Duration_1b'].rename('Time_1'), on='ID')
df_B = df_B.join(df_time['Duration_2b'].rename('Time_2'), on='ID')
df_B = df_B.join(df_time['Duration_3b'].rename('Time_3'), on='ID')
df_B['Avg_Time'] = df_B['Sum_Time'] / 3

# grupa
df_B['Group'] = 'B'

df_B['idx'] = range(40, 80)
df_B.set_index('idx', inplace=True)
df_B

,ID,Total_fix_withCom_1,Total_fix_withCom_2,Total_fix_withCom_3,Sum_TotalFix_WithCom,Avg_TotalFix_WithCom,Total_fix_withoutCom_1,Total_fix_withoutCom_2,Total_fix_withoutCom_3,Sum_TotalFix_WithoutCom,...,Diff_1,Diff_2,Diff_3,Avg_Diff,Sum_Time,Time_1,Time_2,Time_3,Avg_Time,Group
idx,,,,,,,,,,,,,,,,,,,,,
40,46,7042,46754,20848,74644,24881.333333,6867,43937,14873,65677,...,3,2,3,2.666667,329950,96628,113378,119944,109983.333333,B
41,37,26989,58738,73708,159435,53145.000000,22539,55638,69684,147861,...,1,3,4,2.666667,201307,33598,72130,95579,67102.333333,B
42,23,8407,15578,11124,35109,11703.000000,7082,12861,7624,27567,...,1,2,1,1.333333,112862,33632,48148,31082,37620.666667,B
43,40,17878,32215,20964,71057,23685.666667,15570,30090,18622,64282,...,3,4,4,3.666667,114612,30882,48398,35332,38204.000000,B
44,19,25599,35440,106682,167721,55907.000000,21024,31823,101016,153863,...,2,3,4,3.000000,197823,30282,42581,124960,65941.000000,B
45,31,12958,14764,26466,54188,18062.666667,9741,9964,13408,33113,...,2,2,3,2.333333,74796,20499,21232,33065,24932.000000,B
46,38,33300,13217,50543,97060,32353.333333,24276,10368,45144,79788,...,2,1,4,2.333333,172842,44748,30799,97295,57614.000000,B
47,35,11015,6584,22039,39638,13212.666667,11015,6109,19247,36371,...,2,1,4,2.333333,73413,17849,12116,43448,24471.000000,B
48,42,39868,22378,36414,98660,32886.666667,34427,17637,34214,86278,...,1,1,2,1.333333,147776,58347,34215,55214,49258.666667,B


In [88]:
group = 'A'
cols = df_indep.columns
df_indep_merge = df_indep.copy()
for col in cols:
    df_indep_merge[f'Group_{col}'] = df_indep_merge[col].apply(lambda x: f'{group}_{x}')

In [89]:
df_A = df_A.join(df_indep_merge, on='ID')
df_A

,ID,Total_fix_withCom_1,Total_fix_withCom_2,Total_fix_withCom_3,Sum_TotalFix_WithCom,Avg_TotalFix_WithCom,Total_fix_withoutCom_1,Total_fix_withoutCom_2,Total_fix_withoutCom_3,Sum_TotalFix_WithoutCom,...,Group_MAT,Group_POL,Group_HIST,Group_Typ_uczenia,Group_Quest1_before,Group_Quest2_before,Group_Quest1_after,Group_Quest2_after,Group_is_change_1,Group_is_change_2
idx,,,,,,,,,,,,,,,,,,,,,
0,46,4799,10809,9274,24882,8294.000000,3133,4576,7682,15391,...,A_NO_VAL,A_NO_VAL,A_NO_VAL,A_Brak dominującego,A_B,A_mapa,A_A,A_mapa,A_True,A_False
1,37,36009,21199,37874,95082,31694.000000,36009,20967,36079,93055,...,A_NO_VAL,A_NO_VAL,A_NO_VAL,A_Wzrokowiec,A_B,A_wykres,A_B,A_mapa,A_False,A_True
2,23,15724,2400,5808,23932,7977.333333,13774,842,2750,17366,...,A_VAL,A_NO_VAL,A_NO_VAL,A_Kinestetyk,A_B,A_tabela,A_B,A_wykres,A_False,A_True
3,40,43426,7501,13058,63985,21328.333333,42868,6718,9741,59327,...,A_NO_VAL,A_NO_VAL,A_NO_VAL,A_Słuchowiec,A_B,A_mapa,A_A,A_mapa,A_True,A_False
4,19,35214,9624,37404,82242,27414.000000,33214,7183,33022,73419,...,A_VAL,A_NO_VAL,A_NO_VAL,A_Wzrokowiec,A_B,A_mapa,A_B,A_tabela,A_False,A_True
5,31,34481,26914,29965,91360,30453.333333,26340,18965,15083,60388,...,A_NO_VAL,A_VAL,A_NO_VAL,A_Wzrokowiec,A_A,A_mapa,A_A,A_mapa,A_False,A_False
6,38,32123,2442,24498,59063,19687.666667,31338,1767,24498,57603,...,A_NO_VAL,A_NO_VAL,A_NO_VAL,A_Wzrokowiec,A_B,A_wykres,A_B,A_wykres,A_False,A_False
7,35,82120,8255,14701,105076,35025.333333,79503,6547,12410,98460,...,A_VAL,A_NO_VAL,A_NO_VAL,A_Brak dominującego,A_A,A_tabela,A_B,A_wykres,A_True,A_True
8,42,41152,11949,28752,81853,27284.333333,37827,9432,23268,70527,...,A_NO_VAL,A_NO_VAL,A_NO_VAL,A_Wzrokowiec,A_B,A_mapa,A_B,A_wykres,A_False,A_True


In [90]:
group = 'B'
cols = df_indep.columns
df_indep_merge = df_indep.copy()
for col in cols:
    df_indep_merge[f'Group_{col}'] = df_indep_merge[col].apply(lambda x: f'{group}_{x}')

In [91]:
df_B = df_B.join(df_indep_merge, on='ID')
df_B

,ID,Total_fix_withCom_1,Total_fix_withCom_2,Total_fix_withCom_3,Sum_TotalFix_WithCom,Avg_TotalFix_WithCom,Total_fix_withoutCom_1,Total_fix_withoutCom_2,Total_fix_withoutCom_3,Sum_TotalFix_WithoutCom,...,Group_MAT,Group_POL,Group_HIST,Group_Typ_uczenia,Group_Quest1_before,Group_Quest2_before,Group_Quest1_after,Group_Quest2_after,Group_is_change_1,Group_is_change_2
idx,,,,,,,,,,,,,,,,,,,,,
40,46,7042,46754,20848,74644,24881.333333,6867,43937,14873,65677,...,B_NO_VAL,B_NO_VAL,B_NO_VAL,B_Brak dominującego,B_B,B_mapa,B_A,B_mapa,B_True,B_False
41,37,26989,58738,73708,159435,53145.000000,22539,55638,69684,147861,...,B_NO_VAL,B_NO_VAL,B_NO_VAL,B_Wzrokowiec,B_B,B_wykres,B_B,B_mapa,B_False,B_True
42,23,8407,15578,11124,35109,11703.000000,7082,12861,7624,27567,...,B_VAL,B_NO_VAL,B_NO_VAL,B_Kinestetyk,B_B,B_tabela,B_B,B_wykres,B_False,B_True
43,40,17878,32215,20964,71057,23685.666667,15570,30090,18622,64282,...,B_NO_VAL,B_NO_VAL,B_NO_VAL,B_Słuchowiec,B_B,B_mapa,B_A,B_mapa,B_True,B_False
44,19,25599,35440,106682,167721,55907.000000,21024,31823,101016,153863,...,B_VAL,B_NO_VAL,B_NO_VAL,B_Wzrokowiec,B_B,B_mapa,B_B,B_tabela,B_False,B_True
45,31,12958,14764,26466,54188,18062.666667,9741,9964,13408,33113,...,B_NO_VAL,B_VAL,B_NO_VAL,B_Wzrokowiec,B_A,B_mapa,B_A,B_mapa,B_False,B_False
46,38,33300,13217,50543,97060,32353.333333,24276,10368,45144,79788,...,B_NO_VAL,B_NO_VAL,B_NO_VAL,B_Wzrokowiec,B_B,B_wykres,B_B,B_wykres,B_False,B_False
47,35,11015,6584,22039,39638,13212.666667,11015,6109,19247,36371,...,B_VAL,B_NO_VAL,B_NO_VAL,B_Brak dominującego,B_A,B_tabela,B_B,B_wykres,B_True,B_True
48,42,39868,22378,36414,98660,32886.666667,34427,17637,34214,86278,...,B_NO_VAL,B_NO_VAL,B_NO_VAL,B_Wzrokowiec,B_B,B_mapa,B_B,B_wykres,B_False,B_True


In [92]:
df = pd.concat([df_A, df_B])
df

,ID,Total_fix_withCom_1,Total_fix_withCom_2,Total_fix_withCom_3,Sum_TotalFix_WithCom,Avg_TotalFix_WithCom,Total_fix_withoutCom_1,Total_fix_withoutCom_2,Total_fix_withoutCom_3,Sum_TotalFix_WithoutCom,...,Group_MAT,Group_POL,Group_HIST,Group_Typ_uczenia,Group_Quest1_before,Group_Quest2_before,Group_Quest1_after,Group_Quest2_after,Group_is_change_1,Group_is_change_2
idx,,,,,,,,,,,,,,,,,,,,,
0,46,4799,10809,9274,24882,8294.000000,3133,4576,7682,15391,...,A_NO_VAL,A_NO_VAL,A_NO_VAL,A_Brak dominującego,A_B,A_mapa,A_A,A_mapa,A_True,A_False
1,37,36009,21199,37874,95082,31694.000000,36009,20967,36079,93055,...,A_NO_VAL,A_NO_VAL,A_NO_VAL,A_Wzrokowiec,A_B,A_wykres,A_B,A_mapa,A_False,A_True
2,23,15724,2400,5808,23932,7977.333333,13774,842,2750,17366,...,A_VAL,A_NO_VAL,A_NO_VAL,A_Kinestetyk,A_B,A_tabela,A_B,A_wykres,A_False,A_True
3,40,43426,7501,13058,63985,21328.333333,42868,6718,9741,59327,...,A_NO_VAL,A_NO_VAL,A_NO_VAL,A_Słuchowiec,A_B,A_mapa,A_A,A_mapa,A_True,A_False
4,19,35214,9624,37404,82242,27414.000000,33214,7183,33022,73419,...,A_VAL,A_NO_VAL,A_NO_VAL,A_Wzrokowiec,A_B,A_mapa,A_B,A_tabela,A_False,A_True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,44,21474,46064,68222,135760,45253.333333,17849,43595,60348,121792,...,B_VAL,B_NO_VAL,B_NO_VAL,B_Brak dominującego,B_B,B_mapa,B_B,B_tekst,B_False,B_True
76,18,6022,67052,17367,90441,30147.000000,6022,64710,11976,82708,...,B_VAL,B_NO_VAL,B_NO_VAL,B_Wzrokowiec,B_B,B_mapa,B_A,B_wykres,B_True,B_True
77,22,33600,23700,53587,110887,36962.333333,28642,16009,41862,86513,...,B_NO_VAL,B_NO_VAL,B_NO_VAL,B_Brak dominującego,B_B,B_mapa,B_A,B_mapa,B_True,B_False


In [93]:
df.to_csv('/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/analizy_wieloczynnikowe.csv')

Zbiory:
- mapa A vs mapa B

In [31]:
# grafiki A
# fiksacje
# całkowita długość
df_A = df_f_1a.filter(regex='Total_map').sum(axis=1).rename('Total_fix_1').reset_index()
df_A = df_A.join(df_f_2a.filter(regex='Total_map').sum(axis=1).rename('Total_fix_2'), on='ID')
df_A = df_A.join(df_f_3a.filter(regex='Total_map').sum(axis=1).rename('Total_fix_3'), on='ID')
df_A['Sum_TotalFix'] = df_A[['Total_fix_1', 'Total_fix_2', 'Total_fix_3']].sum(axis=1)
df_A['Avg_TotalFix'] = df_A[['Total_fix_1', 'Total_fix_2', 'Total_fix_3']].mean(axis=1)

# średnia długość
df_A = df_A.join(df_f_1a.filter(regex='Avg_map').sum(axis=1).rename('Avg_fix_1'), on='ID')
df_A = df_A.join(df_f_2a.filter(regex='Avg_map').sum(axis=1).rename('Avg_fix_2'), on='ID')
df_A = df_A.join(df_f_3a.filter(regex='Avg_map').sum(axis=1).rename('Avg_fix_3'), on='ID')
df_A['Sum_AvgFix'] = df_A[['Avg_fix_1', 'Avg_fix_2', 'Avg_fix_3']].sum(axis=1)
df_A['Avg_AvgFix'] = df_A[['Avg_fix_1', 'Avg_fix_2', 'Avg_fix_3']].mean(axis=1)

# liczba
df_A = df_A.join(df_f_1a.filter(regex='Num_map').sum(axis=1).rename('Num_fix_1'), on='ID')
df_A = df_A.join(df_f_2a.filter(regex='Num_map').sum(axis=1).rename('Num_fix_2'), on='ID')
df_A = df_A.join(df_f_3a.filter(regex='Num_map').sum(axis=1).rename('Num_fix_3'), on='ID')
df_A['Sum_NumFix'] = df_A[['Num_fix_1', 'Num_fix_2', 'Num_fix_3']].sum(axis=1)
df_A['Avg_NumFix'] = df_A[['Num_fix_1', 'Num_fix_2', 'Num_fix_3']].mean(axis=1)

# średnia wielkość źrenicy
df_A = df_A.join(df_f_1a.filter(regex='Avg_pupil_map').sum(axis=1).rename('Avg_pupil_1'), on='ID')
df_A = df_A.join(df_f_2a.filter(regex='Avg_pupil_map').sum(axis=1).rename('Avg_pupil_2'), on='ID')
df_A = df_A.join(df_f_3a.filter(regex='Avg_pupil_map').sum(axis=1).rename('Avg_pupil_3'), on='ID')
df_A['Avg_AvgPupil'] = df_A[['Avg_pupil_1', 'Avg_pupil_2', 'Avg_pupil_3']].mean(axis=1)

# wizyty
# całkowita długość
df_A = df_A.join(df_v_1a.filter(regex='Total_map').sum(axis=1).rename('Total_visit_1'), on='ID')
df_A = df_A.join(df_v_2a.filter(regex='Total_map').sum(axis=1).rename('Total_visit_2'), on='ID')
df_A = df_A.join(df_v_3a.filter(regex='Total_map').sum(axis=1).rename('Total_visit_3'), on='ID')
df_A['Sum_TotalVisit'] = df_A[['Total_visit_1', 'Total_visit_2', 'Total_visit_3']].sum(axis=1)
df_A['Avg_TotalVisit'] = df_A[['Total_visit_1', 'Total_visit_2', 'Total_visit_3']].mean(axis=1)

# średnia długość
df_A = df_A.join(df_v_1a.filter(regex='Avg_map').sum(axis=1).rename('Avg_visit_1'), on='ID')
df_A = df_A.join(df_v_2a.filter(regex='Avg_map').sum(axis=1).rename('Avg_visit_2'), on='ID')
df_A = df_A.join(df_v_3a.filter(regex='Avg_map').sum(axis=1).rename('Avg_visit_3'), on='ID')
df_A['Sum_AvgVisit'] = df_A[['Avg_visit_1', 'Avg_visit_2', 'Avg_visit_3']].sum(axis=1)
df_A['Avg_AvgVisit'] = df_A[['Avg_visit_1', 'Avg_visit_2', 'Avg_visit_3']].mean(axis=1)

# liczba
df_A = df_A.join(df_v_1a.filter(regex='Num_map').sum(axis=1).rename('Num_visit_1'), on='ID')
df_A = df_A.join(df_v_2a.filter(regex='Num_map').sum(axis=1).rename('Num_visit_2'), on='ID')
df_A = df_A.join(df_v_3a.filter(regex='Num_map').sum(axis=1).rename('Num_visit_3'), on='ID')
df_A['Sum_NumVisit'] = df_A[['Num_visit_1', 'Num_visit_2', 'Num_visit_3']].sum(axis=1)
df_A['Avg_NumVisit'] = df_A[['Num_visit_1', 'Num_visit_2', 'Num_visit_3']].mean(axis=1)

# grupa
df_A['Group'] = 'A'

df_A['idx'] = range(0, 40)
df_A.set_index('idx', inplace=True)
df_A

,ID,Total_fix_1,Total_fix_2,Total_fix_3,Sum_TotalFix,Avg_TotalFix,Avg_fix_1,Avg_fix_2,Avg_fix_3,Sum_AvgFix,...,Avg_visit_2,Avg_visit_3,Sum_AvgVisit,Avg_AvgVisit,Num_visit_1,Num_visit_2,Num_visit_3,Sum_NumVisit,Avg_NumVisit,Group
idx,,,,,,,,,,,,,,,,,,,,,
0,46,3133,4576,7682,15391,5130.333333,104.0,114.0,115.0,333.0,...,5908.0,24815.0,42443.0,14147.666667,2,3,2,7,2.333333,A
1,37,36009,20967,36079,93055,31018.333333,187.0,214.0,228.0,629.0,...,2074.0,8631.0,17531.0,5843.666667,7,12,5,24,8.000000,A
2,23,13774,842,2750,17366,5788.666667,98.0,94.0,110.0,302.0,...,4775.0,14399.0,44555.0,14851.666667,3,1,1,5,1.666667,A
3,40,42868,6718,9741,59327,19775.666667,213.0,232.0,207.0,652.0,...,2703.0,2425.0,16133.0,5377.666667,5,3,5,13,4.333333,A
4,19,33214,7183,33022,73419,24473.000000,242.0,266.0,303.0,811.0,...,2897.0,12288.0,24714.0,8238.000000,4,3,3,10,3.333333,A
5,31,26340,18965,15083,60388,20129.333333,204.0,216.0,193.0,613.0,...,3244.0,3027.0,11579.0,3859.666667,6,7,6,19,6.333333,A
6,38,31338,1767,24498,57603,19201.000000,162.0,104.0,154.0,420.0,...,1737.0,22503.0,50192.0,16730.666667,2,2,2,6,2.000000,A
7,35,79503,6547,12410,98460,32820.000000,209.0,205.0,207.0,621.0,...,2727.0,3834.0,28552.0,9517.333333,5,3,4,12,4.000000,A
8,42,37827,9432,23268,70527,23509.000000,156.0,141.0,174.0,471.0,...,4167.0,6635.0,21957.0,7319.000000,5,4,5,14,4.666667,A


In [30]:
# grafiki A
# fiksacje
# całkowita długość
df_B = df_f_1b.filter(regex='Total_map').sum(axis=1).rename('Total_fix_1').reset_index()
df_B = df_B.join(df_f_2b.filter(regex='Total_map').sum(axis=1).rename('Total_fix_2'), on='ID')
df_B = df_B.join(df_f_3b.filter(regex='Total_map').sum(axis=1).rename('Total_fix_3'), on='ID')
df_B['Sum_TotalFix'] = df_B[['Total_fix_1', 'Total_fix_2', 'Total_fix_3']].sum(axis=1)
df_B['Avg_TotalFix'] = df_B[['Total_fix_1', 'Total_fix_2', 'Total_fix_3']].mean(axis=1)

# średnia długość
df_B = df_B.join(df_f_1b.filter(regex='Avg_map').sum(axis=1).rename('Avg_fix_1'), on='ID')
df_B = df_B.join(df_f_2b.filter(regex='Avg_map').sum(axis=1).rename('Avg_fix_2'), on='ID')
df_B = df_B.join(df_f_3b.filter(regex='Avg_map').sum(axis=1).rename('Avg_fix_3'), on='ID')
df_B['Sum_AvgFix'] = df_B[['Avg_fix_1', 'Avg_fix_2', 'Avg_fix_3']].sum(axis=1)
df_B['Avg_AvgFix'] = df_B[['Avg_fix_1', 'Avg_fix_2', 'Avg_fix_3']].mean(axis=1)

# liczba
df_B = df_B.join(df_f_1b.filter(regex='Num_map').sum(axis=1).rename('Num_fix_1'), on='ID')
df_B = df_B.join(df_f_2b.filter(regex='Num_map').sum(axis=1).rename('Num_fix_2'), on='ID')
df_B = df_B.join(df_f_3b.filter(regex='Num_map').sum(axis=1).rename('Num_fix_3'), on='ID')
df_B['Sum_NumFix'] = df_B[['Num_fix_1', 'Num_fix_2', 'Num_fix_3']].sum(axis=1)
df_B['Avg_NumFix'] = df_B[['Num_fix_1', 'Num_fix_2', 'Num_fix_3']].mean(axis=1)

# średnia wielkość źrenicy
df_B = df_B.join(df_f_1b.filter(regex='Avg_pupil_map').sum(axis=1).rename('Avg_pupil_1'), on='ID')
df_B = df_B.join(df_f_2b.filter(regex='Avg_pupil_map').sum(axis=1).rename('Avg_pupil_2'), on='ID')
df_B = df_B.join(df_f_3b.filter(regex='Avg_pupil_map').sum(axis=1).rename('Avg_pupil_3'), on='ID')
df_B['Avg_AvgPupil'] = df_B[['Avg_pupil_1', 'Avg_pupil_2', 'Avg_pupil_3']].mean(axis=1)

# wizyty
# całkowita długość
df_B = df_B.join(df_v_1b.filter(regex='Total_map').sum(axis=1).rename('Total_visit_1'), on='ID')
df_B = df_B.join(df_v_2b.filter(regex='Total_map').sum(axis=1).rename('Total_visit_2'), on='ID')
df_B = df_B.join(df_v_3b.filter(regex='Total_map').sum(axis=1).rename('Total_visit_3'), on='ID')
df_B['Sum_TotalVisit'] = df_B[['Total_visit_1', 'Total_visit_2', 'Total_visit_3']].sum(axis=1)
df_B['Avg_TotalVisit'] = df_B[['Total_visit_1', 'Total_visit_2', 'Total_visit_3']].mean(axis=1)

# średnia długość
df_B = df_B.join(df_v_1b.filter(regex='Avg_map').sum(axis=1).rename('Avg_visit_1'), on='ID')
df_B = df_B.join(df_v_2b.filter(regex='Avg_map').sum(axis=1).rename('Avg_visit_2'), on='ID')
df_B = df_B.join(df_v_3b.filter(regex='Avg_map').sum(axis=1).rename('Avg_visit_3'), on='ID')
df_B['Sum_AvgVisit'] = df_B[['Avg_visit_1', 'Avg_visit_2', 'Avg_visit_3']].sum(axis=1)
df_B['Avg_AvgVisit'] = df_B[['Avg_visit_1', 'Avg_visit_2', 'Avg_visit_3']].mean(axis=1)

# liczba
df_B = df_B.join(df_v_1b.filter(regex='Num_map').sum(axis=1).rename('Num_visit_1'), on='ID')
df_B = df_B.join(df_v_2b.filter(regex='Num_map').sum(axis=1).rename('Num_visit_2'), on='ID')
df_B = df_B.join(df_v_3b.filter(regex='Num_map').sum(axis=1).rename('Num_visit_3'), on='ID')
df_B['Sum_NumVisit'] = df_B[['Num_visit_1', 'Num_visit_2', 'Num_visit_3']].sum(axis=1)
df_B['Avg_NumVisit'] = df_B[['Num_visit_1', 'Num_visit_2', 'Num_visit_3']].mean(axis=1)

# grupa
df_B['Group'] = 'B'

df_B['idx'] = range(0, 40)
df_B.set_index('idx', inplace=True)
df_B

,ID,Total_fix_1,Total_fix_2,Total_fix_3,Sum_TotalFix,Avg_TotalFix,Avg_fix_1,Avg_fix_2,Avg_fix_3,Sum_AvgFix,...,Avg_visit_2,Avg_visit_3,Sum_AvgVisit,Avg_AvgVisit,Num_visit_1,Num_visit_2,Num_visit_3,Sum_NumVisit,Avg_NumVisit,Group
idx,,,,,,,,,,,,,,,,,,,,,
0,46,4967,14032,4833,23832,7944.000000,110.0,124.0,105.0,339.0,...,8071.0,2186.0,12596.0,4198.666667,10,4,8,22,7.333333,B
1,37,333,32590,23473,56396,18798.666667,167.0,236.0,217.0,620.0,...,3091.0,9952.0,13485.0,4495.000000,1,12,3,16,5.333333,B
2,23,350,3208,2317,5875,1958.333333,117.0,100.0,145.0,362.0,...,1658.0,2066.0,4174.0,1391.333333,1,6,2,9,3.000000,B
3,40,1992,11099,7890,20981,6993.666667,199.0,171.0,202.0,572.0,...,1983.0,2596.0,5373.0,1791.000000,3,8,4,15,5.000000,B
4,19,958,16916,7258,25132,8377.333333,240.0,260.0,213.0,713.0,...,2676.0,1164.0,4352.0,1450.666667,2,7,7,16,5.333333,B
5,31,9624,9414,8250,27288,9096.000000,163.0,214.0,176.0,553.0,...,4706.0,2255.0,14719.0,4906.333333,2,3,5,10,3.333333,B
6,38,2250,7836,8814,18900,6300.000000,205.0,154.0,140.0,499.0,...,1368.0,1447.0,3488.0,1162.666667,4,9,11,24,8.000000,B
7,35,167,2050,8607,10824,3608.000000,167.0,186.0,166.0,519.0,...,585.0,2133.0,2885.0,961.666667,1,4,6,11,3.666667,B
8,42,2833,4357,15349,22539,7513.000000,189.0,168.0,185.0,542.0,...,685.0,2371.0,4250.0,1416.666667,3,8,9,20,6.666667,B


In [32]:
df = pd.concat([df_A, df_B])
df

,ID,Total_fix_1,Total_fix_2,Total_fix_3,Sum_TotalFix,Avg_TotalFix,Avg_fix_1,Avg_fix_2,Avg_fix_3,Sum_AvgFix,...,Avg_visit_2,Avg_visit_3,Sum_AvgVisit,Avg_AvgVisit,Num_visit_1,Num_visit_2,Num_visit_3,Sum_NumVisit,Avg_NumVisit,Group
idx,,,,,,,,,,,,,,,,,,,,,
0,46,3133,4576,7682,15391,5130.333333,104.0,114.0,115.0,333.0,...,5908.0,24815.0,42443.0,14147.666667,2,3,2,7,2.333333,A
1,37,36009,20967,36079,93055,31018.333333,187.0,214.0,228.0,629.0,...,2074.0,8631.0,17531.0,5843.666667,7,12,5,24,8.000000,A
2,23,13774,842,2750,17366,5788.666667,98.0,94.0,110.0,302.0,...,4775.0,14399.0,44555.0,14851.666667,3,1,1,5,1.666667,A
3,40,42868,6718,9741,59327,19775.666667,213.0,232.0,207.0,652.0,...,2703.0,2425.0,16133.0,5377.666667,5,3,5,13,4.333333,A
4,19,33214,7183,33022,73419,24473.000000,242.0,266.0,303.0,811.0,...,2897.0,12288.0,24714.0,8238.000000,4,3,3,10,3.333333,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,44,0,3017,14291,17308,5769.333333,0.0,189.0,265.0,454.0,...,1808.0,2030.0,3838.0,1279.333333,0,2,8,10,3.333333,B
36,18,708,27739,5543,33990,11330.000000,236.0,187.0,173.0,596.0,...,3443.0,1713.0,5897.0,1965.666667,1,14,5,20,6.666667,B
37,22,3308,9018,15231,27557,9185.666667,158.0,225.0,190.0,573.0,...,2323.0,3548.0,7565.0,2521.666667,3,5,6,14,4.666667,B


In [33]:
df.to_csv('/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/mapaA_vs_mapaB.csv')

Zbiór mapa vs tabela vs tekst vs wykres

In [64]:
# mapa
var = 'map'
map_f_1 = df_f_1b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_fix_1', f'Avg_{var}': 'Avg_fix_1', f'Num_{var}':'Num_fix_1', f'Avg_pupil_{var}':'Avg_pupil_1'}).reset_index()
map_v_1 = df_v_1b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_visit_1', f'Avg_{var}': 'Avg_visit_1', f'Num_{var}':'Num_visit_1'})
map_1 = map_f_1.join(map_v_1, on='ID')

map_f_2 = df_f_2b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_fix_2', f'Avg_{var}': 'Avg_fix_2', f'Num_{var}':'Num_fix_2', f'Avg_pupil_{var}':'Avg_pupil_2'}).reset_index()
map_v_2 = df_v_2b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_visit_2', f'Avg_{var}': 'Avg_visit_2', f'Num_{var}':'Num_visit_2'})
map_2 = map_f_2.join(map_v_2, on='ID').set_index('ID')

map_f_3 = df_f_3b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_fix_3', f'Avg_{var}': 'Avg_fix_3', f'Num_{var}':'Num_fix_3', f'Avg_pupil_{var}':'Avg_pupil_3'}).reset_index()
map_v_3 = df_v_3b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_visit_3', f'Avg_{var}': 'Avg_visit_3', f'Num_{var}':'Num_visit_3'})
map_3 = map_f_3.join(map_v_3, on='ID').set_index('ID')

map = map_1.join(map_2, on='ID')
map = map.join(map_3, on='ID')
map['Group'] = 'mapa'

# tabela
var = 'tab'
tab_f_1 = df_f_1b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_fix_1', f'Avg_{var}': 'Avg_fix_1', f'Num_{var}':'Num_fix_1', f'Avg_pupil_{var}':'Avg_pupil_1'}).reset_index()
tab_v_1 = df_v_1b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_visit_1', f'Avg_{var}': 'Avg_visit_1', f'Num_{var}':'Num_visit_1'})
tab_1 = tab_f_1.join(tab_v_1, on='ID')

tab_f_2 = df_f_2b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_fix_2', f'Avg_{var}': 'Avg_fix_2', f'Num_{var}':'Num_fix_2', f'Avg_pupil_{var}':'Avg_pupil_2'}).reset_index()
tab_v_2 = df_v_2b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_visit_2', f'Avg_{var}': 'Avg_visit_2', f'Num_{var}':'Num_visit_2'})
tab_2 = tab_f_2.join(tab_v_2, on='ID').set_index('ID')

tab_f_3 = df_f_3b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_fix_3', f'Avg_{var}': 'Avg_fix_3', f'Num_{var}':'Num_fix_3', f'Avg_pupil_{var}':'Avg_pupil_3'}).reset_index()
tab_v_3 = df_v_3b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_visit_3', f'Avg_{var}': 'Avg_visit_3', f'Num_{var}':'Num_visit_3'})
tab_3 = tab_f_3.join(tab_v_3, on='ID').set_index('ID')

tab = tab_1.join(tab_2, on='ID')
tab = tab.join(tab_3, on='ID')
tab['Group'] = 'tabela'

# tekst
var = 'txt'
txt_f_1 = df_f_1b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_fix_1', f'Avg_{var}': 'Avg_fix_1', f'Num_{var}':'Num_fix_1', f'Avg_pupil_{var}':'Avg_pupil_1'}).reset_index()
txt_v_1 = df_v_1b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_visit_1', f'Avg_{var}': 'Avg_visit_1', f'Num_{var}':'Num_visit_1'})
txt_1 = txt_f_1.join(txt_v_1, on='ID')

txt_f_2 = df_f_2b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_fix_2', f'Avg_{var}': 'Avg_fix_2', f'Num_{var}':'Num_fix_2', f'Avg_pupil_{var}':'Avg_pupil_2'}).reset_index()
txt_v_2 = df_v_2b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_visit_2', f'Avg_{var}': 'Avg_visit_2', f'Num_{var}':'Num_visit_2'})
txt_2 = txt_f_2.join(txt_v_2, on='ID').set_index('ID')

txt_f_3 = df_f_3b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_fix_3', f'Avg_{var}': 'Avg_fix_3', f'Num_{var}':'Num_fix_3', f'Avg_pupil_{var}':'Avg_pupil_3'}).reset_index()
txt_v_3 = df_v_3b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_visit_3', f'Avg_{var}': 'Avg_visit_3', f'Num_{var}':'Num_visit_3'})
txt_3 = txt_f_3.join(txt_v_3, on='ID').set_index('ID')

txt = txt_1.join(txt_2, on='ID')
txt = txt.join(txt_3, on='ID')
txt['Group'] = 'tekst'

# wykres
var = 'plt'
plt_f_1 = df_f_1b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_fix_1', f'Avg_{var}': 'Avg_fix_1', f'Num_{var}':'Num_fix_1', f'Avg_pupil_{var}':'Avg_pupil_1'}).reset_index()
plt_v_1 = df_v_1b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_visit_1', f'Avg_{var}': 'Avg_visit_1', f'Num_{var}':'Num_visit_1'})
plt_1 = plt_f_1.join(plt_v_1, on='ID')

plt_f_2 = df_f_2b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_fix_2', f'Avg_{var}': 'Avg_fix_2', f'Num_{var}':'Num_fix_2', f'Avg_pupil_{var}':'Avg_pupil_2'}).reset_index()
plt_v_2 = df_v_2b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_visit_2', f'Avg_{var}': 'Avg_visit_2', f'Num_{var}':'Num_visit_2'})
plt_2 = plt_f_2.join(plt_v_2, on='ID').set_index('ID')

plt_f_3 = df_f_3b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_fix_3', f'Avg_{var}': 'Avg_fix_3', f'Num_{var}':'Num_fix_3', f'Avg_pupil_{var}':'Avg_pupil_3'}).reset_index()
plt_v_3 = df_v_3b.filter(regex=f'^.*_{var}').rename(columns={f'Total_{var}':'Total_visit_3', f'Avg_{var}': 'Avg_visit_3', f'Num_{var}':'Num_visit_3'})
plt_3 = plt_f_3.join(plt_v_3, on='ID').set_index('ID')

plt = plt_1.join(plt_2, on='ID')
plt = plt.join(plt_3, on='ID')
plt['Group'] = 'wykres'

In [78]:
df = pd.concat([map, plt, tab, txt]).reset_index().drop(columns='index')
df['Sum_Total_fix'] = df.filter(regex='Total_fix').sum(axis=1)
df['Avg_Total_fix'] = df.filter(regex='Total_fix').mean(axis=1)
df['Sum_Total_visit'] = df.filter(regex='Total_visit').sum(axis=1)
df['Avg_Total_visit'] = df.filter(regex='Total_visit').mean(axis=1)

df['Sum_Avg_fix'] = df.filter(regex='Avg_fix').sum(axis=1)
df['Avg_Avg_fix'] = df.filter(regex='Avg_fix').mean(axis=1)
df['Sum_Avg_visit'] = df.filter(regex='Avg_visit').sum(axis=1)
df['Avg_Avg_visit'] = df.filter(regex='Avg_visit').mean(axis=1)

df['Sum_Num_fix'] = df.filter(regex='Num_fix').sum(axis=1)
df['Avg_Num_fix'] = df.filter(regex='Num_fix').mean(axis=1)
df['Sum_Num_visit'] = df.filter(regex='Num_visit').sum(axis=1)
df['Avg_Num_visit'] = df.filter(regex='Num_visit').mean(axis=1)

df['Avg_AvgPupil'] = df.filter(regex='Avg_pupil').mean(axis=1)

In [80]:
df.to_csv('/Users/martasolarz/Studies/Thesis/Master_thesis/prepare_datasets/sets/map_tab_plt_txt.csv')

Test dla płci + grupy

In [117]:
df['Płeć_Group'] = df['Płeć'] + '_' + df['Group']
df[['Płeć', 'Group', 'Płeć_Group', 'Corr_1']]

,Płeć,Group,Płeć_Group,Corr_1
idx,,,,
0,M,A,M_A,1
1,M,A,M_A,1
2,M,A,M_A,1
3,K,A,K_A,1
4,K,A,K_A,1
...,...,...,...,...
75,K,B,K_B,1
76,M,B,M_B,1
77,K,B,K_B,1


In [136]:
ALPHA = 0.05

Wieloczynnikowa analiza regresji 

In [151]:
# Płeć + Grupa oddzielnie
md = MixedLM.from_formula('Corr_1 ~ C(Group) + C(Płeć)', data=df, groups=df["ID"])
mdf = md.fit()
print(mdf.summary())

p_values = mdf.pvalues[1:-1]
names = mdf.model.exog_names[1:]
for i, p_val in enumerate(p_values):
    print('-------------------')
    print(f"Zmienna: {names[i]}, P-value: {p_val}")
    make_decision(p_val, ALPHA)

         Mixed Linear Model Regression Results
Model:             MixedLM Dependent Variable: Corr_1  
No. Observations:  80      Method:             REML    
No. Groups:        40      Scale:              0.0500  
Min. group size:   2       Log-Likelihood:     -17.1190
Max. group size:   2       Converged:          Yes     
Mean group size:   2.0                                 
-------------------------------------------------------
              Coef. Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.823    0.059 14.047 0.000  0.708  0.938
C(Group)[T.B] 0.050    0.050  1.000 0.317 -0.048  0.148
C(Płeć)[T.M]  0.123    0.081  1.511 0.131 -0.037  0.282
Group Var     0.040    0.093                           

-------------------
Zmienna: C(Group)[T.B], P-value: 0.31730761965954013
Brak podstaw do odrzucenia H0.
-------------------
Zmienna: C(Płeć)[T.M], P-value: 0.13085351164490155
Brak podstaw do odrzucenia H0.


In [152]:
# Płeć + Grupa razem
md = MixedLM.from_formula('Corr_1 ~ Płeć_Group', data=df, groups=df["ID"])
mdf = md.fit()
print(mdf.summary())

p_values = mdf.pvalues[1:-1]
names = mdf.model.exog_names[1:]
for i, p_val in enumerate(p_values):
    print('-------------------')
    print(f"Zmienna: {names[i]}, P-value: {p_val}")
    make_decision(p_val, ALPHA)

           Mixed Linear Model Regression Results
Model:               MixedLM  Dependent Variable:  Corr_1  
No. Observations:    80       Method:              REML    
No. Groups:          40       Scale:               0.0467  
Min. group size:     2        Log-Likelihood:      -16.6976
Max. group size:     2        Converged:           Yes     
Mean group size:     2.0                                   
-----------------------------------------------------------
                  Coef. Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept         0.783    0.062 12.660 0.000  0.661  0.904
Płeć_Group[T.K_B] 0.130    0.064  2.047 0.041  0.006  0.255
Płeć_Group[T.M_A] 0.217    0.095  2.292 0.022  0.032  0.403
Płeć_Group[T.M_B] 0.159    0.095  1.672 0.094 -0.027  0.344
Group Var         0.041    0.097                           

-------------------
Zmienna: Płeć_Group[T.K_B], P-value: 0.040692588684053836
Istnieją podstawy do odrzucenia 

Wieloczynnikowa analiza wariancji

In [153]:
# Płeć i grupa oddzielnie
model = ols('Corr_1 ~ C(Group) + C(Płeć)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

p_values = anova_table['PR(>F)'][:-1]
names = anova_table.index[:-1]
for i, p_val in enumerate(p_values):
    print('-------------------')
    print(f"Zmienna: {names[i]}, P-value: {p_val}")
    make_decision(p_val, ALPHA)

            sum_sq    df         F    PR(>F)
C(Group)  0.050000   1.0  0.561603  0.455898
C(Płeć)   0.294629   1.0  3.309295  0.072777
Residual  6.855371  77.0       NaN       NaN
-------------------
Zmienna: C(Group), P-value: 0.4558983461662266
Brak podstaw do odrzucenia H0.
-------------------
Zmienna: C(Płeć), P-value: 0.07277659180179832
Brak podstaw do odrzucenia H0.


In [154]:
# Płeć i grupa razem
model = ols('Corr_1 ~ Płeć_Group', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

p_values = anova_table['PR(>F)'][:-1]
names = anova_table.index[:-1]
for i, p_val in enumerate(p_values):
    print('-------------------')
    print(f"Zmienna: {names[i]}, P-value: {p_val}")
    make_decision(p_val, ALPHA)

              sum_sq    df         F    PR(>F)
Płeć_Group  0.519693   3.0  1.970801  0.125472
Residual    6.680307  76.0       NaN       NaN
-------------------
Zmienna: Płeć_Group, P-value: 0.12547208145524164
Brak podstaw do odrzucenia H0.
